План:

1. Импорты
2. Функции загрузки инфы
3. Предобработка

## Импорты

In [1]:
from enum import IntFlag # Для декодирования битовых флагов

import datetime as dt
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import MetaTrader5 as mt5


# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# выведем данные о пакете MetaTrader5
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


In [2]:
%%html
<style>
table {float:left}
</style>

**Тикер**

https://www.moex.com/ru/derivatives/equity/indices/  
Ссылка, чтобы узнать текущий тикер фьюча на РТС

In [3]:
ticker = 'RIM2'

Нам необходимо получить строки, в которых одновременно будет находиться инфа об одной сделке (тике) + информация о лимитных ордерах 20 бидов и 20 асков или сколько даст мт5.

Для тренировки нужно собрать исторические данные о тиках и данные о стакане за тот же день так, чтобы можно было наиболее точно совместить момент (время) сделки с состоянием стакана в этот момент.

Также будем пробовать использовать сингулярный спектральный анализ: https://www.kaggle.com/code/jdarcy/introducing-ssa-for-time-series-decomposition/notebook

https://pyts.readthedocs.io/en/stable/auto_examples/decomposition/plot_ssa.html

## Функции загрузки данных

В этом разделе содержаться функции загрузки информации о сделках (ticks) и стакане (market_book). Но предобработку будем отрабатывать на уже сохраненных тестовых данных. Этот раздел пока можно пропустить.

TODO:
* Обернуть инициализацию и шатдаун мт5 в функцию
* Собрать информацию о стакане в течении торгового дня

### Запуск mt5

In [ ]:
# Запуск мт5
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
else:
    print(mt5.last_error())

### Функция получения тиков

In [ ]:
def get_ticks(symbol,days_before=1, copy_tick_flag=mt5.COPY_TICKS_ALL):
    '''
    Берет с мт5 тики за период от (текущей даты минус 
    days_before, начало дня) до текущей даты (даже если это выходной).
    
    Столбец 'time' перевод в пандавский дейттайм
    Столбец 'time_msc' тоже самое, только с учетом мс.
    
    Если запускать в выходные то может вернуть наны, 
    нужно это учитывать определяя days_before
    
    '''
    date_to = dt.datetime.now(dt.timezone.utc)
    date_from = date_to - dt.timedelta(days = days_before)
    date_from = date_from.replace(hour=0, minute=0, second=0,microsecond=0)
    

    df_rates = pd.DataFrame(mt5.copy_ticks_range(symbol,
                                                 date_from,
                                                 date_to,
                                                 copy_tick_flag))
    
    df_rates['time'] = pd.to_datetime(df_rates['time'], unit='s',utc=True)
    df_rates['time_msc'] = pd.to_datetime(df_rates['time_msc'],unit='ms',utc=True)
    df_rates["symbol"] = symbol
#     df_rates = df_rates.set_index('time_msc',drop=True)

    return df_rates
 

# df = get_ticks(symbol=ticker,days_before=1)

# display(df.head())
# example = df.tail(1)
# display(example)

Описание полученных из мт5 данных.

| Column | Description |
| :------|:------------ |
|**'time'**|Время последнего обновления цен
|**'bid'**|Текущая цена Bid
|**'ask'**|Текущая цена Ask
|**'last'**|Текущая цена последней сделки (Last)
|**'volume'**|Объем для текущей цены Last
|**'time_msc'**|Время последнего обновления цен в миллисекундах
|**'flags'**|Флаги тиков
|**'volume_real'**|Объем для текущей цены Last c повышенной точностью

### Функция получения инфы со стакана

In [ ]:
# Эту штуку нужно как-нибудь запустить на день
# С начала торгового дня до конца
columns = ['type','price','volume','volume_dbl']

df_mb = pd.DataFrame()

mt5.market_book_add(ticker)

counter = 0

while dt.datetime.now().hour < 13:
#     print(counter)
    df_market_book = pd.DataFrame(mt5.market_book_get(ticker),columns=columns)
    df_market_book['time'] = dt.datetime.now()
    df_market_book['batch_number'] = counter
    df_mb = pd.concat([df_mb,df_market_book])
    counter += 1    
    time.sleep(0.05)
    
mt5.market_book_release(ticker)

df_mb

In [ ]:
# df_mb.info()
# df_mb.to_csv('data/market_book.csv')
# df_mb['time'].describe(datetime_is_numeric=True)

In [ ]:
# Ф-ция разового запроса стакана
# Здесь каждый раз идет регистрация "сбора инфы о стакане" с помощью
# mt5.market_book_add(symbol),
# а затем отзыв
# mt5.market_book_release(symbol)
def get_market_book(symbol):
    mt5.market_book_add(symbol)
    columns = ['type','price','volume','volume_dbl']
    df_market_book = pd.DataFrame(mt5.market_book_get(symbol),columns=columns)
    df_market_book['time'] = dt.datetime.now()

    mt5.market_book_release(symbol)
    return df_market_book

# get_market_book(ticker)

### Шатдаун подключения

In [ ]:
# завершим подключение к терминалу MetaTrader 5
mt5.shutdown()

## Предобработка time&sales (ticks)

Используем ранее сохраненный датасет.

In [5]:
# df.to_csv('data/test_data_ticks.csv')
def load_test_data():
    df = pd.read_csv('data/test_data_ticks.csv',index_col=0)
    df['time'] = pd.to_datetime(df['time'])
    df['time_msc'] = pd.to_datetime(df['time_msc'])
    df = df[df['last'] != 0]
    df = df.set_index('time_msc')
    cols = ['bid','ask','last','volume','flags']
    df = df[cols]
    return df

df = load_test_data()
df.head()

,bid,ask,last,volume,flags
time_msc,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,312
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,312
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,6
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,312
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,344


Описание сохраненных данных

| Column | Description |
| :------|:------------ |
|**'time_msc'**|Время последнего обновления цен в миллисекундах
|**'bid'**|Текущая цена Bid
|**'ask'**|Текущая цена Ask
|**'last'**|Текущая цена последней сделки (Last)
|**'volume'**|Объем для текущей цены Last
|**'flags'**|Флаги тиков


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 382442 entries, 2022-05-05 10:00:00.254000+00:00 to 2022-05-06 18:23:07.643000+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   bid     382442 non-null  float64
 1   ask     382442 non-null  float64
 2   last    382442 non-null  float64
 3   volume  382442 non-null  int64  
 4   flags   382442 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 17.5 MB


* Пропусков нет.

### Обработка bitmask флагов

**Информация о flags**

| Flag | Value | Description |
| :------|:------------: | :---------|
|TICK_FLAG_BID|2|a tick has changed a Bid price
|TICK_FLAG_ASK|4|a tick has changed an Ask price
|TICK_FLAG_LAST|8|a tick has changed the last deal price
|TICK_FLAG_VOLUME|16|a tick has changed a volume
|TICK_FLAG_BUY|32|a tick is a result of a buy deal
|TICK_FLAG_SELL|64|a tick is a result of a sell deal

In [7]:
df['flags'].value_counts()

344    65374
312    64358
88     63761
56     62903
2      46459
4      44870
6      34717
Name: flags, dtype: int64

Создадим столбцы с каждым флагом.

In [8]:
# Создадим класс с флагами
class tick_flag(IntFlag):
    bid_changed = 2
    ask_changed = 4
    last_changed = 8
    volume_changed = 16
    buy_done = 32
    sell_done = 64
    # Добавим еще 3 флага, потому что был замечен 256, но не описан в документации
    unknown_1 = 128
    unknown_2 = 256 
    unknown_3 = 512

In [9]:
cols_ = ['bid_changed','ask_changed','last_changed',
         'volume_changed','buy_done','sell_done',
         'unknown_1','unknown_2','unknown_3']

for col,flag in zip(cols_,tick_flag):
    df[col] = (df['flags'] & flag) == flag

In [10]:
df.head(1)

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,312,False,False,True,True,True,False,False,True,False


Изменим содержание столбца `'flags'` для наглядности.  
Переведем в текстовые флаги.

In [11]:
def flag_decode_to_str(x):
    str_ = ''
    for flag in tick_flag:
        if (x & flag) == flag:
            str_+=str(flag.name) + " "
    return str_


# flag_decode_to_str(312)
df['flags'] = df['flags'].apply(flag_decode_to_str)
df.head(1)

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False


In [12]:
df.head()

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,bid_changed ask_changed,True,True,False,False,False,False,False,False,False
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,last_changed volume_changed sell_done unknown_2,False,False,True,True,False,True,False,True,False


### Создание столбца со спредами

In [13]:
df['spread'] = df['ask'] - df['bid']
df.head()

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3,spread
time_msc,,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,bid_changed ask_changed,True,True,False,False,False,False,False,False,False,760.0
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,last_changed volume_changed sell_done unknown_2,False,False,True,True,False,True,False,True,False,760.0


In [14]:
df['spread'].describe()

count    382442.000000
mean         30.365206
std          20.477696
min          10.000000
25%          20.000000
50%          30.000000
75%          40.000000
max        1040.000000
Name: spread, dtype: float64

## Предобработка market book

Загрузим тестовые данные.

In [15]:
df_mb = pd.read_csv('data/market_book.csv',index_col=0)
df_mb.head()

,type,price,volume,volume_dbl,time,batch_number
0,1,107970.0,1,1.0,2022-05-11 11:48:13.695893,0
1,1,107960.0,2,2.0,2022-05-11 11:48:13.695893,0
2,1,107950.0,1,1.0,2022-05-11 11:48:13.695893,0
3,1,107940.0,1,1.0,2022-05-11 11:48:13.695893,0
4,1,107930.0,2,2.0,2022-05-11 11:48:13.695893,0


In [18]:
df_mb.head(40)

,type,price,volume,volume_dbl,time,batch_number
0,1,107970.0,1,1.0,2022-05-11 11:48:13.695893,0
1,1,107960.0,2,2.0,2022-05-11 11:48:13.695893,0
2,1,107950.0,1,1.0,2022-05-11 11:48:13.695893,0
3,1,107940.0,1,1.0,2022-05-11 11:48:13.695893,0
4,1,107930.0,2,2.0,2022-05-11 11:48:13.695893,0
5,1,107920.0,1,1.0,2022-05-11 11:48:13.695893,0
6,1,107900.0,3,3.0,2022-05-11 11:48:13.695893,0
7,1,107890.0,1,1.0,2022-05-11 11:48:13.695893,0
8,1,107880.0,6,6.0,2022-05-11 11:48:13.695893,0
9,1,107870.0,2,2.0,2022-05-11 11:48:13.695893,0


В одном запросе 40 строк, т.е. 20 бидов и 20 асков.

Проверим, справедливо ли это ко всем запросам.

In [23]:
(df_mb.groupby('batch_number')['price'].count() != 40).sum()

0

Справедливо ко всем.

Проверим во всех ли случаях `'volume'` равен `'volume_dbl'`.

In [24]:
(df_mb['volume'] != df_mb['volume_dbl']).sum()

0

Во всех.

Удалим `'volume_dbl'`.

In [25]:
df_mb = df_mb.drop(columns=['volume_dbl'])

In [26]:
df_mb.columns

Index(['type', 'price', 'volume', 'time', 'batch_number'], dtype='object')

In [54]:
mb_cols = np.arange(20,-21,step=-1)
mb_cols = np.delete(mb_cols, np.where(mb_cols==0))
mb_cols

array([ 20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,   9,   8,
         7,   6,   5,   4,   3,   2,   1,  -1,  -2,  -3,  -4,  -5,  -6,
        -7,  -8,  -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19,
       -20])

In [60]:
# df_mb.pivot(index='time',columns=['type'])
# df_mb['delta_ask'] = df.groupby(['batch'])
# df_mb.head(2)
df_test = df_mb.head(40)
# df_test = df_test.T
# df_test.columns = mb_cols
# df_test.T

In [65]:
pd.concat([df_test['price'],df_test['volume']]).reset_index(drop=True)

0     107970.0
1     107960.0
2     107950.0
3     107940.0
4     107930.0
        ...   
75         1.0
76         2.0
77         6.0
78         4.0
79         7.0
Length: 80, dtype: float64

Сделаем цикл по групбаю по батч_намбер или дате, в котором индексом будет дата, а информация о цене и объеме сделаем в одну строку.

In [87]:
test = pd.DataFrame(pd.concat([df_test['price'],df_test['volume']])).T
test.index = [date]
test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
2022-05-11 11:48:13.695893,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,2.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0


In [100]:
# new_df_mb = pd.DataFrame()

# for date,data in df_mb.groupby('time'):
#     result = pd.DataFrame(pd.concat([data['price'],data['volume']])).reset_index(drop=True).T
#     result.index = [date]
# #     result = pd.DataFrame(data=pd.concat([data['price'],data['volume']]).reset_index(drop=True).T,index=[date])
#     new_df_mb = pd.concat([new_df_mb,result])

# # new_df_mb.to_csv('data/market_book_T.csv')

new_df_mb = pd.read_csv('data/market_book_T.csv',index_col=0,parse_dates=True)
display(new_df_mb.info())
new_df_mb.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 57532 entries, 2022-05-11 11:48:13.695893 to 2022-05-11 12:59:59.912702
Data columns (total 80 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       57532 non-null  float64
 1   1       57532 non-null  float64
 2   2       57532 non-null  float64
 3   3       57532 non-null  float64
 4   4       57532 non-null  float64
 5   5       57532 non-null  float64
 6   6       57532 non-null  float64
 7   7       57532 non-null  float64
 8   8       57532 non-null  float64
 9   9       57532 non-null  float64
 10  10      57532 non-null  float64
 11  11      57532 non-null  float64
 12  12      57532 non-null  float64
 13  13      57532 non-null  float64
 14  14      57532 non-null  float64
 15  15      57532 non-null  float64
 16  16      57532 non-null  float64
 17  17      57532 non-null  float64
 18  18      57532 non-null  float64
 19  19      57532 non-null  float64
 20  20      57532 non-null 

None

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
2022-05-11 11:48:13.695893,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,2.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0
2022-05-11 11:48:13.747754,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,5.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0
2022-05-11 11:48:13.799217,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,5.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0
2022-05-11 11:48:13.850972,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,5.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0
2022-05-11 11:48:13.902292,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,5.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0


## Объединение тиков со стаканом

In [104]:
df.index.min(),df.index.max()

(Timestamp('2022-05-05 10:00:00.254000+0000', tz='UTC'),
 Timestamp('2022-05-06 18:23:07.643000+0000', tz='UTC'))

Возьмем только 05.05 из тиков, а индекс стакана переделаем из 11.05 в 05.05.

In [109]:
df_ticks_test = df.loc["2022-05-05 10:00:00":"2022-05-05 16:00:00"]

In [110]:
df_ticks_test.index.max()

Timestamp('2022-05-05 16:00:00.994000+0000', tz='UTC')

In [116]:
new_df_mb.index = new_df_mb.index - pd.Timedelta(6,'D')

In [117]:
new_df_mb.head(1)

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
2022-05-05 11:48:13.695893,107970.0,107960.0,107950.0,107940.0,107930.0,107920.0,107900.0,107890.0,107880.0,107870.0,...,9.0,32.0,2.0,4.0,4.0,1.0,2.0,6.0,4.0,7.0


Попробуем замерджить по времени с помощью merge_asof

In [125]:
test_full_df = pd.merge_asof(df_ticks_test,new_df_mb,left_index=True,right_index=True,direction='nearest')

In [120]:
test_full_df.columns

Index(['bid', 'ask', 'last', 'volume', 'flags', 'bid_changed', 'ask_changed',
       'last_changed', 'volume_changed', 'buy_done', 'sell_done', 'unknown_1',
       'unknown_2', 'unknown_3', 'spread', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
       '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79'],
      dtype='object')

In [126]:
test_full_df = test_full_df[test_full_df['0'].notna()]

In [127]:
test_full_df.count()

bid       146491
ask       146491
last      146491
volume    146491
flags     146491
           ...  
75        146491
76        146491
77        146491
78        146491
79        146491
Length: 95, dtype: int64

In [124]:
new_df_mb.count()

0     57532
1     57532
2     57532
3     57532
4     57532
      ...  
75    57532
76    57532
77    57532
78    57532
79    57532
Length: 80, dtype: int64

### Исследование признаков

In [ ]:
df.info()

In [ ]:
df['bid'].value_counts(normalize=True)

In [ ]:
for col in df.columns:
    display(col)
    display(df[col].value_counts(normalize=True).iloc[:5])
    

In [ ]:
df['ask'].nunique()

In [ ]:
df['last'].nunique()

# Фьючерс РТС тики

**Задача:**

Обучить модель машинного обучения на информации о каждой сделке по фьючерсам РТС, используя:
* Инфу о самих тиках
* Инфу о стакане

Таргет:

В течении следующих 5 минут цена дошла до тейкпрофита и не дошла до стоплосса.

Предварительно: 3 к 1, профит к стоплоссу.

Необходимо решить как использовать инфу о тиках и стакане, т.к. инфу за тики можно получить в виде истории, а инфу о стакане можно получить только на текущий момент, поэтому невозможно со 100% точностью к каждому тику присоединить информацию о состоянии стакана в момент сделки.

Нужно делать какие-то упрощения. Например, саккумулировать инфу о стакане в течении торгового дня, собирая ее раз в секунду, или меньше - как позволит система. Присоединить эту инфу к инфе о тиках 1 к 1 с возможностью появления пропусков, а затем эти пропуски заполнить вперед. 

Какие модели использовать, я пока не знаю. Нужно будет разобраться с моделями таймсериес и разложением на спектр. Еще помню была какая-то новость, в которой было о модели машинного обучения для таймсериес очень эффективной.

И вообще-то собрав данные я могу их как-то проанализировать, чтобы найти характеристики в которых таргет = 1. Характеристики рынка.

Тогда мои две главные задачи:

1. Сделать функцию для построения таргета
2. Написать функцию сбора данных о стакане

# Повторяющийся кусок инициализации

### Функция расстановки таргета

https://pandas.pydata.org/docs/user_guide/window.html#rolling-window-endpoints

Тестирование алгоритма проставление таргета.

У этой цены в следующие 5 минут цена поднимется на 2?
Мы тут проставили желаемый таргет руками и дальше плясали от него.


<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/window.html#:~:text=We%20can%20also%20achieve%20this%20by%20using%20slicing%2C%20applying%20rolling%20aggregation%2C%20and%20then%20flipping%20the%20result%20as%20shown%20in%20example%20below%3A">Ссылка на прием, чтобы окно смотрело в будущее</a>


In [ ]:
df_time = pd.DataFrame({'last': [1,2,3,4,5,6,7,8,7,6,5,4,3,4,5,6,7,8,7,6],
                        'target_true': [1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0]
                       },
                       index = pd.date_range("2018-01-01", periods=20, freq="1min"))


# from pandas.api.indexers import FixedForwardWindowIndexer
# indexer = FixedForwardWindowIndexer(window_size=3)
# df_time['target_test'] = df_time['last'].shift().rolling(indexer).max()


# Этот прием с двумя разворотами нужен, чтобы окно смотрело в будущее
# Ссылка выше
df_time['max_price_next_5mins'] = df_time['last'][::-1].rolling("5min",closed='left').max()[::-1]


df_time['target_proposed'] = df_time['max_price_next_5mins'] >= (df_time['last'] + 2)
df_time

Теперь проверим у этой цены в ближайшие 5 минут цена опустится ниже текущей на 1?

In [ ]:
df_time = pd.DataFrame({'last': [1,2,3,4,5,6,7,8,7,6,5,4,3,4,5,6,7,8,7,6],
                        'target_true': [0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0]
                       },
                       index = pd.date_range("2018-01-01", periods=20, freq="1min"))



# Этот прием с двумя разворотами нужен, чтобы окно смотрело в будущее
# Ссылка выше
df_time['min_price_next_5mins'] = df_time['last'][::-1].rolling("5min",closed='left').min()[::-1]


df_time['target_proposed'] = df_time['min_price_next_5mins'] <= (df_time['last'] - 1)
df_time

<!-- Совместим эти два метода, чтобы построить таргет следующим образом:  
В следующие 5 минут цена дойдет то +2 раньше, чем опустится до -1? -->

Стоит ли делать такой таргет, чтобы он отслеживал какое событие произошло раньше цена поднялась на +2 или опустилась на -1?  
Или же расчитывать вероятность по каждому правилу отдельно и принимать решение о сделке в совокупности вероятностей?  
Так можно еще будет настраивать порог.

Пока пойду по второму пути.

Напишем функцию.

In [ ]:
def create_buy_target(df,take_profit_value,stop_loss_value):
    
    df = df.copy()
    
    max_price_next_5mins = df['last'][::-1].rolling("5min",closed='left').max()[::-1]
    df['target_tp'] = max_price_next_5mins >= (df['last'] + take_profit_value)
    
    min_price_next_5mins = df['last'][::-1].rolling("5min",closed='left').min()[::-1]
    df['target_sl'] = min_price_next_5mins <= (df['last'] - stop_loss_value)
    
    return df

df = create_buy_target(df,100,30)

In [ ]:
df

In [ ]:
df['2022-05-05 10:00:00+00:00':'2022-05-05 10:05:00+00:00']

In [ ]:
df.iloc[10:20]

In [ ]:
# mt5.market_book_add(ticker)
# display(mt5.market_book_get(ticker))
# mt5.market_book_release(ticker)

In [ ]:
mt5.market_book_get(ticker).to_dict()

In [ ]:
# market_book = get_market_book(ticker).to_dict()
# market_book

In [ ]:
# df=None
# counter = 0
# while not df.empty or df != None:
#     df = df.append(get_market_book(ticker))
#     counter += 1
#     if counter > 5:
#         break
# else:
#     mt5.last_error()
# display(dt.datetime.now())
# # df = df.set_index(['time','price'])
# df

### Функция получения свечек за период

# Давай сделаем получение инфы со стакана и объединим его с дф тиков